<a href="https://colab.research.google.com/github/bystrowska/idiom-paraphrasing/blob/main/idiom_paraphrasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>